In [1]:
import os
import copy
import json
import numpy as np
from tqdm import tqdm

In [2]:
WEIGHTED = True # True：重み付き平均されたベクトルを用いる（カテゴリベクトル・都道府県ベクトル）False：用いない
data_dir = "jalan_review_aligned_v3/"

#
len(os.listdir(data_dir))

1883

# データの読み込み

In [3]:
if WEIGHTED:
    category_dir = f"{data_dir}weighted_categories/"
else:    
    category_dir = f"{data_dir}categories/"
category_paths = [category_dir + CP for CP in os.listdir(category_dir)]

categories = [C.split(".")[0] for C in os.listdir(category_dir)]
category_datas = dict()
for C, CP in zip(categories, category_paths):
    with open(CP, "r") as f:
        category_datas[C] = {
            "vector" : np.array(json.load(f)["vector"])
        }

#
categories

['スキー・スノーボード',
 'ダム',
 'ビーチ・海水浴場',
 '公園・庭園',
 '動物園・植物園',
 '博物館',
 '史跡・名所巡り',
 '城郭',
 '展望台・展望施設',
 '山岳',
 '文化史跡・遺跡',
 '歴史的建造物',
 '水族館',
 '滝・渓谷',
 '町並み',
 '神社・神宮・寺院',
 '近代建築']

In [4]:
if WEIGHTED:
    prefecture_dir = f"{data_dir}/weighted_prefectures/"
else:
    prefecture_dir = f"{data_dir}/prefectures/"
prefecture_paths = [prefecture_dir + PP for PP in os.listdir(prefecture_dir)][1:]

prefectures = [P.split(".")[0] for P in os.listdir(prefecture_dir)][1:]
prefecture_datas = dict()
for P, PP in zip(prefectures, prefecture_paths):
    with open(PP, "r") as f:
        prefecture_datas[P] = {
            "vector" : np.array(json.load(f)["vector"])
        }

#
prefectures

['京都府',
 '佐賀県',
 '兵庫県',
 '北海道',
 '千葉県',
 '和歌山県',
 '埼玉県',
 '大分県',
 '大阪府',
 '奈良県',
 '宮城県',
 '宮崎県',
 '富山県',
 '山口県',
 '山形県',
 '山梨県',
 '岐阜県',
 '岡山県',
 '岩手県',
 '島根県',
 '広島県',
 '徳島県',
 '愛媛県',
 '愛知県',
 '新潟県',
 '東京都',
 '栃木県',
 '沖縄県',
 '滋賀県',
 '熊本県',
 '石川県',
 '神奈川県',
 '福井県',
 '福岡県',
 '福島県',
 '秋田県',
 '群馬県',
 '茨城県',
 '長崎県',
 '長野県',
 '青森県',
 '静岡県',
 '香川県',
 '高知県',
 '鳥取県',
 '鹿児島県']

In [5]:
spot_datas = dict()
spot_dir = f"{data_dir}"
spot_paths = [spot_dir + SP for SP in os.listdir(spot_dir)]

for SP in tqdm(spot_paths):
    if "json" not in SP:
        continue

    with open(SP, "r") as f:
        a = json.load(f)
    name = a["name"]
    category = a["category"]
    prefecture = a["prefecture"]
    vector = a["vector"]

    spot_datas[name] = {
        "name" : name,
        "category" : category,
        "prefecture" : prefecture,
        "vector" : np.array(vector)
    }
    
# 
len(spot_datas.keys())

100%|██████████| 1883/1883 [00:09<00:00, 192.09it/s]


1875

In [6]:
def get_cossim(vector_A, vector_B):
    return np.dot(vector_A, vector_B.T) / (np.linalg.norm(vector_A) * np.linalg.norm(vector_B))

def get_ranking(vector, datas):
    tmp = list()
    for K in datas.keys():
        D = copy.deepcopy(datas[K])
        cossim = get_cossim(D["vector"], vector)
        D["cossim"] = cossim
        tmp.append(D)
    return list(reversed(sorted(tmp, key = lambda x : x["cossim"])))

def print_result(datas_with_cossim):
    for i, D in enumerate(datas_with_cossim):
        print(i + 1, D["name"], D["cossim"], D["category"], D["prefecture"])

def get_only_condition(datas, prefecture = None, category = None):
    tmp = dict()
    for K in datas.keys():
        flag = True
        if prefecture != None and datas[K]["prefecture"] != prefecture:
            flag = False
        if category != None and datas[K]["category"] != category:
            flag = False
        
        if flag:
            tmp[K] = datas[K]
    
    return tmp

# ベクトル演算

In [7]:
a = category_datas["動物園・植物園"]["vector"]
b = prefecture_datas["広島県"]["vector"]

tmp = get_ranking(a + b, spot_datas)
print_result(tmp)

1 栗林公園 0.9392317585563853 公園・庭園 香川県
2 修善寺虹の郷 0.9383647712830506 公園・庭園 静岡県
3 懐古園 0.9378606104361791 文化史跡・遺跡 長野県
4 岡山後楽園 0.9258611582092964 公園・庭園 岡山県
5 日立市かみね公園 0.9234701438443325 公園・庭園 茨城県
6 高岡古城公園（城跡） 0.9199302648821522 文化史跡・遺跡 富山県
7 水前寺成趣園 0.9150813030746751 公園・庭園 熊本県
8 江の島サムエル・コッキング苑 0.9093522765323617 動物園・植物園 神奈川県
9 淡路島国営明石海峡公園 0.9069309905007195 公園・庭園 兵庫県
10 生田緑地 0.9054658298201537 公園・庭園 神奈川県
11 三景園 0.9038194970158314 公園・庭園 広島県
12 縮景園 0.9035074138178589 公園・庭園 広島県
13 手柄山中央公園 0.9025383177473166 公園・庭園 兵庫県
14 兼六園（石川県金沢市） 0.9025180251456254 公園・庭園 石川県
15 三溪園 0.9020925896884878 公園・庭園 神奈川県
16 千光寺公園 0.9015106002515834 公園・庭園 広島県
17 平和台公園（宮崎県宮崎市） 0.9006039243385701 公園・庭園 宮崎県
18 ときわ公園 0.9004456836440301 動物園・植物園 山口県
19 御船山楽園 0.9002111420173333 公園・庭園 佐賀県
20 須磨浦公園 0.9001228403252037 公園・庭園 兵庫県
21 西山公園（福井県鯖江市） 0.8999384623273646 公園・庭園 福井県
22 小室山公園 0.8989155718273012 公園・庭園 静岡県
23 とっとり花回廊 0.8983949660108544 動物園・植物園 鳥取県
24 道後公園 0.8979310147924986 公園・庭園 愛媛県
25 フラワーパークかごしま 0.8978332135236108 動物園・植物園 鹿児島

In [8]:
a = spot_datas["新宿御苑"]["vector"]
b = prefecture_datas["東京都"]["vector"]
c = prefecture_datas["大阪府"]["vector"]

tmp = get_ranking(a - 10*b + 10*c, spot_datas)
print_result(tmp)

1 彩湖・道満グリーンパーク 0.6511990467809 公園・庭園 埼玉県
2 淀川河川公園 0.647026620258574 公園・庭園 京都府
3 久宝寺緑地 0.6461881889581015 公園・庭園 大阪府
4 浜寺公園 0.6435890981953398 公園・庭園 大阪府
5 矢橋帰帆島公園 0.6418327538313502 公園・庭園 滋賀県
6 辰口丘陵公園 0.6378259600817615 公園・庭園 石川県
7 大泉緑地 0.6292769989614858 公園・庭園 大阪府
8 服部緑地 0.6285510258125175 公園・庭園 大阪府
9 千葉県立柏の葉公園 0.6270215760574981 公園・庭園 千葉県
10 深北緑地 0.6267631894556726 公園・庭園 大阪府
11 せんなん里海公園 0.6259902938155831 公園・庭園 大阪府
12 しあわせの村 0.6230604355187235 公園・庭園 兵庫県
13 寝屋川公園 0.618226302324423 公園・庭園 大阪府
14 山田池公園 0.6175425040225914 公園・庭園 大阪府
15 蜻蛉池公園 0.6170470747243887 公園・庭園 大阪府
16 播磨中央公園 0.6141764271857686 公園・庭園 兵庫県
17 扇町公園（大阪府大阪市） 0.6039562559530631 公園・庭園 大阪府
18 ネーブルパーク 0.6030628224827095 公園・庭園 茨城県
19 引地台公園 0.601899162021919 公園・庭園 神奈川県
20 長沼フートピア公園 0.5933030506141813 公園・庭園 宮城県
21 そうか公園 0.5926194151466305 公園・庭園 埼玉県
22 花博記念公園鶴見緑地 0.5919373168070499 公園・庭園 大阪府
23 呉ポートピアパーク 0.5909906746750745 公園・庭園 広島県
24 神奈川県立辻堂海浜公園 0.5885247553068326 公園・庭園 神奈川県
25 こども自然公園 0.5868283091565214 公園・庭園 神奈川県
26 兵庫県立赤穂海浜公園 0.

In [20]:
a = spot_datas["新宿御苑"]["vector"]
b = prefecture_datas["東京都"]["vector"]
c = prefecture_datas["大阪府"]["vector"]

tmp = get_ranking(a - 5*b + 5*c, 
                  get_only_condition(spot_datas, prefecture = '大阪府', category = '公園・庭園'))
print_result(tmp)

1 浜寺公園 0.8171920164268882 公園・庭園 大阪府
2 久宝寺緑地 0.81222963325215 公園・庭園 大阪府
3 服部緑地 0.8113571819739823 公園・庭園 大阪府
4 山田池公園 0.807270345879717 公園・庭園 大阪府
5 大泉緑地 0.8042203759500922 公園・庭園 大阪府
6 深北緑地 0.7955296171704165 公園・庭園 大阪府
7 寝屋川公園 0.7941016463642392 公園・庭園 大阪府
8 花博記念公園鶴見緑地 0.793976382715107 公園・庭園 大阪府
9 蜻蛉池公園 0.7857056665731288 公園・庭園 大阪府
10 扇町公園（大阪府大阪市） 0.7735527874350269 公園・庭園 大阪府
11 長居公園 0.7637841259113081 公園・庭園 大阪府
12 せんなん里海公園 0.7634817033480594 公園・庭園 大阪府
13 城北公園 0.7453437181960726 公園・庭園 大阪府
14 万博記念公園 0.7406625397348607 公園・庭園 大阪府
15 大仙公園 0.7303402823226355 公園・庭園 大阪府
16 中之島公園（大阪府大阪市） 0.7217777984490933 公園・庭園 大阪府
17 大浜公園（大阪府堺市） 0.7171328350742081 公園・庭園 大阪府
18 大阪城公園 0.7130230212727656 公園・庭園 大阪府
19 住吉公園（大阪市住之江区） 0.6986409801809516 公園・庭園 大阪府
20 りんくう公園 0.6960883449973928 公園・庭園 大阪府
21 五月山公園 0.6803659982217347 公園・庭園 大阪府
22 万博記念公園・日本庭園 0.6688886126266358 公園・庭園 大阪府
23 府民の森［くろんど・ほしだ・むろいけ・なるかわ・くさか・ぬかた・みずのみ・ちはや・ほりご園地］ 0.6560624220846304 公園・庭園 大阪府
24 大阪城西の丸庭園 0.6529898888215538 公園・庭園 大阪府
25 毛馬桜之宮公園 0.63509

In [22]:
a = spot_datas["桜島"]["vector"]
b = prefecture_datas["鹿児島県"]["vector"]
c = prefecture_datas["北海道"]["vector"]

tmp = get_ranking(a - 2*b + 2*c, 
                  get_only_condition(spot_datas, prefecture = '北海道', category = '山岳'))
print_result(tmp)

1 藻岩山（北海道札幌市） 0.8105463588998191 山岳 北海道
2 函館山 0.8008900219176984 山岳 北海道
3 ニセコアンヌプリ 0.7984310850840326 山岳 北海道
4 昭和新山 0.7869168453466856 山岳 北海道
5 大倉山 0.7822501446824419 山岳 北海道
6 旭岳 0.759713869043826 山岳 北海道
7 羊蹄山（北海道京極町） 0.7450197944856981 山岳 北海道
8 硫黄山（北海道弟子屈町） 0.6491123559471974 山岳 北海道
9 硫黄山（北海道斜里町） 0.6263321294962173 山岳 北海道
10 石垣山（北海道愛別町） 0.5255222862718611 山岳 北海道


In [10]:
a = spot_datas["海遊館"]["vector"]
b = prefecture_datas["大阪府"]["vector"]
c = prefecture_datas["沖縄県"]["vector"]

tmp = get_ranking(a - b + c, spot_datas)
print_result(tmp)

1 沖縄美ら海水族館 0.9538079707588527 水族館 沖縄県
2 のとじま水族館 0.9232022468734312 水族館 石川県
3 いおワールドかごしま水族館 0.9177445133690864 水族館 鹿児島県
4 海中水族館シードーナツ 0.9161783071276908 水族館 熊本県
5 海遊館 0.9152484466295192 水族館 大阪府
6 アクアマリンふくしま 0.9109629240252194 水族館 福島県
7 横浜・八景島シーパラダイス　アクアリゾーツ 0.9095989933876859 水族館 神奈川県
8 アクアワールド茨城県大洗水族館 0.9082658691843861 水族館 茨城県
9 九十九島水族館海きらら 0.9057386464034043 水族館 長崎県
10 越前松島水族館 0.9049715510416948 水族館 福井県
11 新屋島水族館 0.9028066394066473 水族館 香川県
12 下関市立しものせき水族館・海響館 0.9026953956570731 水族館 山口県
13 大分マリーンパレス水族館「うみたまご」 0.9025414139557378 水族館 大分県
14 マリンワールド海の中道 0.9014521241137465 水族館 福岡県
15 名古屋港水族館 0.8987064758455605 水族館 愛知県
16 新江ノ島水族館 0.898700067463609 水族館 神奈川県
17 浅虫水族館 0.8982128562552055 水族館 青森県
18 男鹿水族館GAO 0.8949377220085923 水族館 秋田県
19 新潟市水族館 マリンピア日本海 0.8917720809661397 水族館 新潟県
20 京急油壺マリンパーク 0.8910397477174935 水族館 神奈川県
21 城崎マリンワールド 0.8896002875589946 水族館 兵庫県
22 あわしまマリンパーク 0.8848503814784959 水族館 静岡県
23 桂浜水族館 0.8844643077349951 水族館 高知県
24 伊勢夫婦岩ふれあい水族館(伊勢シーパラダイス) 0.8839403554921249 水族館 三重県
25 神戸

In [11]:
a = spot_datas["明治神宮"]["vector"]

tmp = get_ranking(10*a, spot_datas)
print_result(tmp)

1 明治神宮 0.9999999999999998 神社・神宮・寺院 東京都
2 井草八幡宮 0.946185542972303 神社・神宮・寺院 東京都
3 橿原神宮 0.9435244454279881 神社・神宮・寺院 奈良県
4 熱田神宮 0.9315330305584262 神社・神宮・寺院 愛知県
5 武蔵一宮氷川神社 0.9312273793058157 神社・神宮・寺院 埼玉県
6 宇佐神宮 0.9229823671116294 神社・神宮・寺院 大分県
7 伊勢神宮内宮（皇大神宮） 0.9224444387796886 神社・神宮・寺院 三重県
8 鹿島神宮 0.922253322252551 神社・神宮・寺院 茨城県
9 霧島神宮 0.9214135168134535 神社・神宮・寺院 鹿児島県
10 宮崎神宮 0.9212474319457067 神社・神宮・寺院 宮崎県
11 安房神社（千葉県館山市） 0.9194978831606733 神社・神宮・寺院 千葉県
12 香取神宮 0.9167983460464184 神社・神宮・寺院 千葉県
13 花園神社（東京都新宿区） 0.9158828611950629 神社・神宮・寺院 東京都
14 北海道神宮 0.9158500301793685 神社・神宮・寺院 北海道
15 榛名神社（群馬県沼田市） 0.915756382654527 神社・神宮・寺院 群馬県
16 阿蘇神社（東京都羽村市） 0.912838605492502 神社・神宮・寺院 東京都
17 大國魂神社（東京都府中市） 0.912785581030223 神社・神宮・寺院 東京都
18 靖国神社 0.912389149940505 神社・神宮・寺院 東京都
19 代々木八幡宮 0.9097490076295918 神社・神宮・寺院 東京都
20 彌彦神社 0.9095740164425322 神社・神宮・寺院 新潟県
21 日枝神社（東京都清瀬市） 0.9082065419942258 神社・神宮・寺院 東京都
22 鹿児島神宮 0.9067947835934093 神社・神宮・寺院 鹿児島県
23 箱根神社 0.9064861351756845 神社・神宮・寺院 神奈川県
24 二宮神社 0.9062123828201785

In [12]:
a = spot_datas["忠犬ハチ公像"]["vector"]
b = prefecture_datas["東京都"]["vector"]
c = prefecture_datas["大阪府"]["vector"]

tmp = get_ranking(a - 10*b + 10*c, spot_datas)
print_result(tmp)

1 彩湖・道満グリーンパーク 0.5202998289090645 公園・庭園 埼玉県
2 矢橋帰帆島公園 0.5100532119026006 公園・庭園 滋賀県
3 淀川河川公園 0.5077966874462599 公園・庭園 京都府
4 せんなん里海公園 0.5048603186392995 公園・庭園 大阪府
5 久宝寺緑地 0.5039823139792046 公園・庭園 大阪府
6 辰口丘陵公園 0.5024141941534598 公園・庭園 石川県
7 扇町公園（大阪府大阪市） 0.4993538322263088 公園・庭園 大阪府
8 浜寺公園 0.49246680545874144 公園・庭園 大阪府
9 深北緑地 0.48837688461639356 公園・庭園 大阪府
10 堀内公園 0.4854466196409786 公園・庭園 愛知県
11 大泉緑地 0.4842246705324 公園・庭園 大阪府
12 長沼フートピア公園 0.4794554254385763 公園・庭園 宮城県
13 蜻蛉池公園 0.478147762298411 公園・庭園 大阪府
14 ネーブルパーク 0.4767212337903827 公園・庭園 茨城県
15 神戸総合運動公園 0.4744173654481418 公園・庭園 兵庫県
16 しあわせの村 0.4732139022924153 公園・庭園 兵庫県
17 服部緑地 0.4712324078166813 公園・庭園 大阪府
18 神奈川県立辻堂海浜公園 0.4673126828602863 公園・庭園 神奈川県
19 寝屋川公園 0.46649351238349984 公園・庭園 大阪府
20 千葉県立柏の葉公園 0.46219589594690913 公園・庭園 千葉県
21 呉ポートピアパーク 0.46162368481377214 公園・庭園 広島県
22 播磨中央公園 0.459048412093494 公園・庭園 兵庫県
23 兵庫県立赤穂海浜公園 0.4585206372159092 公園・庭園 兵庫県
24 大高緑地 0.45831052001288025 公園・庭園 愛知県
25 明石公園（愛知県碧南市） 0.45755194962698403 公園・庭園 愛知県
26 と

In [13]:
a = spot_datas["明治神宮"]["vector"]
b = category_datas["神社・神宮・寺院"]["vector"]
#c = category_datas["史跡・名所巡り"]["vector"]
c = prefecture_datas["東京都"]["vector"]

tmp = get_cossim(a - b , c)
print(tmp)

0.12148463290988905


In [14]:
a = spot_datas["斑尾高原スキー場"]["vector"]
b = category_datas["スキー・スノーボード"]["vector"]
#c = category_datas["史跡・名所巡り"]["vector"]
c = prefecture_datas["新潟県"]["vector"]

tmp = get_cossim(a - b , c)
print(tmp)

0.2787187670409085


In [15]:
a = spot_datas["斑尾高原スキー場"]["vector"]
#b = category_datas["スキー・スノーボード"]["vector"]
#c = category_datas["史跡・名所巡り"]["vector"]
c = prefecture_datas["新潟県"]["vector"]

tmp = get_cossim(a , c)
print(tmp)

0.8509698864188359


In [16]:
a = spot_datas["斑尾高原スキー場"]["vector"]
#b = category_datas["スキー・スノーボード"]["vector"]
#c = category_datas["史跡・名所巡り"]["vector"]
c = prefecture_datas["長野県"]["vector"]

tmp = get_cossim(a , c)
print(tmp)

0.757714759052616


In [17]:
#a = spot_datas["斑尾高原スキー場"]["vector"]
b = category_datas["スキー・スノーボード"]["vector"]
#c = category_datas["史跡・名所巡り"]["vector"]
c = prefecture_datas["北海道"]["vector"]

tmp = get_cossim(b , c)
print(tmp)

0.8598484977996191


In [23]:
a = spot_datas["舞鶴赤れんがパーク"]["vector"]
b = prefecture_datas["京都府"]["vector"]
c = prefecture_datas["東京都"]["vector"]

tmp = get_ranking(a - b + c, spot_datas)
print_result(tmp)

1 舞鶴赤れんがパーク 0.9575779741163064 史跡・名所巡り 京都府
2 横浜赤レンガ倉庫［1号館及び2号館］ 0.8689629058307772 歴史的建造物 神奈川県
3 門司港レトロ地区 0.8476513367744832 町並み 福岡県
4 倉敷アイビースクエア 0.8346392707828156 歴史的建造物 岡山県
5 お台場 0.8342044553247151 町並み 東京都
6 三笠公園 0.8338181668761626 公園・庭園 神奈川県
7 山居倉庫 0.8333288959129287 歴史的建造物 山形県
8 メリケンパーク・ハーバーランド 0.8202423027832256 町並み 兵庫県
9 日本丸メモリアルパーク（旧横浜船渠第1号ドック） 0.8168575864357046 歴史的建造物 神奈川県
10 横浜市開港記念会館 0.8117947962027767 歴史的建造物 神奈川県
11 手柄山中央公園 0.807608418351221 公園・庭園 兵庫県
12 コスモスクエア 0.8030564589075079 町並み 大阪府
13 ぎふ清流里山公園 0.8018387053061795 公園・庭園 岐阜県
14 夢みなとタワー 0.8010222887410345 展望台・展望施設 鳥取県
15 カラコロ工房 0.7992412771449846 歴史的建造物 島根県
16 吉野ヶ里歴史公園 0.7965977571753015 文化史跡・遺跡 佐賀県
17 豆田町 0.7935747385594806 町並み 大分県
18 旧唐津銀行 0.7930885107001887 歴史的建造物 佐賀県
19 岩手銀行赤レンガ館 0.7895239717123882 史跡・名所巡り 岩手県
20 黒壁スクエア 0.7887031416087087 町並み 滋賀県
21 愛・地球博記念公園 0.7881606695647833 公園・庭園 愛知県
22 白壁土蔵群 0.7862482372973039 町並み 鳥取県
23 倉敷美観地区 0.7845177806558283 町並み 岡山県
24 函館市旧イギリス領事館 0.7845170641612964 歴史的建造物 北海道
25 出島（長崎県長崎市